In [1]:
"""
Simple ensembling
1.by weight
2. geomean
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from scipy.stats.mstats import gmean

# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../input/blenditall"]).decode("utf8"))

bidirectional-gru-with-convolution.csv
bi-gru-cnn-poolings.csv
bilstm_conv_preprocess.csv
bilstm_preprocess.csv
capsule-gru_preprocess.csv
corr_blend.csv
global-avg-pool_preprocess.csv
hight_of_blending.csv
lvl0_lgbm_clean_sub.csv
oof-stacking-regime.csv
pooled-gru-fasttext-6c07c9.csv
pooled-gru-fasttext_preprocess.csv
pooled-gru-glove-with-preprocessing.csv
ridge_preprocess.csv
superblend_1.csv
tidy_xgb_glm.csv
toxic-avenger.csv
why-a-such-low-score-with-r-and-keras.csv
wordbatch-fm-ftrl.csv



In [2]:
## collect all results from different models
grucnn = pd.read_csv('../input/blenditall/bi-gru-cnn-poolings.csv')
gruglo = pd.read_csv("../input/blenditall/pooled-gru-glove-with-preprocessing.csv")
ave = pd.read_csv("../input/blenditall/toxic-avenger.csv")
supbl= pd.read_csv('../input/blenditall/superblend_1.csv')
best = pd.read_csv('../input/blenditall/hight_of_blending.csv')
lgbm = pd.read_csv('../input/blenditall/lvl0_lgbm_clean_sub.csv')
wordbtch = pd.read_csv('../input/blenditall/wordbatch-fm-ftrl.csv')
tidy = pd.read_csv('../input/blenditall/tidy_xgb_glm.csv')
fast = pd.read_csv('../input/blenditall/pooled-gru-fasttext-6c07c9.csv')
bilst = pd.read_csv('../input/blenditall/bilstm_conv_preprocess.csv')
oofs = pd.read_csv('../input/blenditall/oof-stacking-regime.csv')
corrbl = pd.read_csv('../input/blenditall/corr_blend.csv')
rkera = pd.read_csv('../input/blenditall/why-a-such-low-score-with-r-and-keras.csv')

lstm_conv = pd.read_csv('../subs/bilstm_conv_preprocess.csv')
lstm_bi = pd.read_csv('../subs/bilstm_preprocess.csv')
gru_capsule = pd.read_csv('../subs/capsule-gru_preprocess.csv')
gru_pool = pd.read_csv('../subs/global-avg-pool_preprocess.csv')
gru_fasttext = pd.read_csv('../subs/pooled-gru-fasttext_preprocess.csv')
ridge = pd.read_csv('../subs/ridge_preprocess.csv')

b1 = best.copy()
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

from sklearn.preprocessing import minmax_scale
for label in labels:
    gru_capsule[label] = minmax_scale(gru_capsule[label])
    gru_pool[label] = minmax_scale(gru_pool[label])
    lstm_bi[label] = minmax_scale(lstm_bi[label])
    lstm_conv[label] = minmax_scale(lstm_conv[label])
    gru_fasttext[label] = minmax_scale(gru_fasttext[label])
    ridge[label] = minmax_scale(ridge[label])

In [3]:
##weights

for i in labels:
    b1[i] = (2 * fast[i]  + 2 * gruglo[i] + grucnn[i] * 4 + ave[i] + supbl[i] * 2 
             + best[i] * 4 +  wordbtch[i] * 2 + lgbm[i] * 2 + 
             tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4 #33
             +1*gru_capsule[i]+2*gru_pool[i]+4*lstm_bi[i]+2*lstm_conv[i]+2*gru_fasttext[i]+ridge[i]) /  45
    
b1.to_csv('blend_it_all_v3.csv', index = False)

In [7]:
###geomean
preds= pd.DataFrame()

for i in labels:
    preds[label] = pd.concat([gru_capsule[i], 
                    gru_pool[i],
                    lstm_bi[i], 
                    lstm_conv[i],
                    gru_fasttext[i],
                    ridge[i],
                    fast[i],gruglo[i],grucnn[i], ave[i] ,supbl[i] , 
                    best[i], wordbtch[i],lgbm[i], 
                    tidy[i] ,bilst[i] , oofs[i],corrbl[i] ]
                      )
    
    b1[label] = preds[label].groupby(level=0).apply(gmean)

b1.to_csv('geoblenditall.csv', index=False)

/usr/local/lib/python3.5/dist-packages/scipy/stats/stats.py:305: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
